In [1]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
from torch_geometric.transforms import FaceToEdge
from torch_geometric.data import Data, DataLoader
import numpy as np
import pandas as pd
from os import path
import pyreadr
from typing import Tuple


class GraphData:
    file_points_asymp = path.join("lv-ortho-modes", "data", "surface_points_ASYMP.RData")
    file_points_mi = path.join("lv-ortho-modes", "data", "surface_points_MI.RData")
    file_face = path.join("lv-ortho-modes", "data", "surface_face.RData")

    df_points_asymp = pyreadr.read_r(file_points_asymp)["X.ASYMP"]
    df_points_mi = pyreadr.read_r(file_points_mi)["X.MI"]
    df_faces = pyreadr.read_r(file_face)["faces"]
    
    points = 2523
    
    def __init__(self, train_x: int=200, train_y: int=200, test_x: int=100, test_y: int=100) -> None:
        self.split_into_train_test(train_x, train_y, test_x, test_y)
        self.normalize_data()
        self.create_loader()
        
    def _split_points(self, df: pd.DataFrame) -> Tuple[pd.DataFrame, ...]:    
        # order: [ED-endo ED-epi ES-endo ES-epi]
        
        
        ed_endo = df.iloc[:, :self.points]
        ed_epi = df.iloc[:, self.points:self.points*2]
        es_endo = df.iloc[:, self.points*2:self.points*3]
        es_epi = df.iloc[:, self.points*3:self.points*4]
        
        return (ed_endo, ed_epi, es_endo, es_epi)
    
    def split_into_train_test(self, train_x: int, train_y: int, test_x: int, test_y: int) -> None:
    
        points = { "ed_endo": {}, "es_endo": {} }
        
        points["ed_endo"]["asymp"], _, points["es_endo"]["asymp"], _ = self._split_points(self.df_points_asymp)
        points["ed_endo"]["mi"], _, points["es_endo"]["mi"], _ = self._split_points(self.df_points_mi)
        
        transformer = FaceToEdge(True)
        
        self.train_samples = []
        self.test_samples = []

        faces = torch.tensor(self.df_faces.values - 1, dtype=torch.long).t().contiguous()
        
        def create_data(start: int, end: int, ed_endo: pd.DataFrame, 
                        es_endo: pd.DataFrame, y: bool, list_: list
                       ) -> None:
            for ed, es in zip(ed_endo.values[start:end], es_endo.values[start:end]):
                pos_ed = torch.tensor(ed.reshape(self.points//3, 3), dtype=torch.float)
                pos_es = torch.tensor(es.reshape(self.points//3, 3), dtype=torch.float)
                data = Data(pos=pos_ed, face=faces, y=y)
                data.x = torch.tensor(np.concatenate((pos_ed, pos_es), axis=1))
                list_.append(transformer(data))
                
        create_data(0, train_x, points["ed_endo"]["asymp"], points["es_endo"]["asymp"], 0, self.train_samples)
        create_data(0, train_y, points["ed_endo"]["mi"], points["es_endo"]["mi"], 1, self.train_samples)
        create_data(train_x, train_x + test_x, points["ed_endo"]["asymp"], points["es_endo"]["asymp"], 0, self.test_samples)
        create_data(train_y, train_y + test_y, points["ed_endo"]["mi"], points["es_endo"]["mi"], 1, self.test_samples)
        
    def normalize_data(self) -> None:
        input_features = torch.cat([data.x for data in self.train_samples], axis=0)
        inputs_mean, inputs_std = torch.mean(input_features), torch.std(input_features)
        
        def normalize_set(dataset: list) -> None:
            for data in dataset:
                data.x = ((data.x - inputs_mean) / inputs_std)
            return dataset
                
        self.train_samples = normalize_set(self.train_samples)
        self.test_samples = normalize_set(self.test_samples)
        
    def create_loader(self) -> None:
        self.train_loader = DataLoader(self.train_samples, batch_size=128, shuffle=True)
        self.test_loader = DataLoader(self.test_samples, batch_size=128, shuffle=True)

In [3]:
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.nn import global_mean_pool

class GCN(torch.nn.Module):
    def __init__(self, hidden_channels, device=device):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(6, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.conv3 = GCNConv(hidden_channels, hidden_channels)
        self.lin = Linear(hidden_channels, 2)

    def forward(self, x, edge_index, batch):
        # 1. Obtain node embeddings 
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = self.conv2(x, edge_index)
        x = x.relu()
        x = self.conv3(x, edge_index)

        # 2. Readout layer
        x = global_mean_pool(x, batch)  # [batch_size, hidden_channels]

        # 3. Apply a final classifier
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin(x)
        
        return x

In [ ]:
graph_data = GraphData()

model = GCN(hidden_channels=64).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()

def train():
    model.train()

    for data in graph_data.train_loader: # Iterate in batches over the training dataset.
        data = data.to(device)
        out = model(data.x, data.edge_index, data.batch)  # Perform a single forward pass.
        loss = criterion(out, data.y)  # Compute the loss.
        loss.backward()  # Derive gradients.
        optimizer.step()  # Update parameters based on gradients.
        optimizer.zero_grad()  # Clear gradients.

def test(loader):
    model.eval()

    correct = 0
    for data in loader:  # Iterate in batches over the training/test dataset.
        data = data.to(device)
        out = model(data.x, data.edge_index, data.batch)  
        pred = out.argmax(dim=1)  # Use the class with highest probability.
        correct += int((pred == data.y).sum())  # Check against ground-truth labels.
    return correct / len(loader.dataset)  # Derive ratio of correct predictions.


for epoch in range(1, 200):
    train()
    train_acc = test(graph_data.train_loader)
    test_acc = test(graph_data.test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}')

Epoch: 001, Train Acc: 0.5575, Test Acc: 0.5650
Epoch: 002, Train Acc: 0.5700, Test Acc: 0.5650
Epoch: 003, Train Acc: 0.5075, Test Acc: 0.5150
Epoch: 004, Train Acc: 0.7100, Test Acc: 0.6750
Epoch: 005, Train Acc: 0.7050, Test Acc: 0.7150
Epoch: 006, Train Acc: 0.8700, Test Acc: 0.8600
Epoch: 007, Train Acc: 0.8675, Test Acc: 0.8550
Epoch: 008, Train Acc: 0.9075, Test Acc: 0.8700
Epoch: 009, Train Acc: 0.9000, Test Acc: 0.8750
Epoch: 010, Train Acc: 0.8575, Test Acc: 0.8450
Epoch: 011, Train Acc: 0.8825, Test Acc: 0.8600
Epoch: 012, Train Acc: 0.8875, Test Acc: 0.8800
Epoch: 013, Train Acc: 0.9025, Test Acc: 0.8800
Epoch: 014, Train Acc: 0.9000, Test Acc: 0.8700
Epoch: 015, Train Acc: 0.9100, Test Acc: 0.8850
Epoch: 016, Train Acc: 0.9075, Test Acc: 0.8850
Epoch: 017, Train Acc: 0.9100, Test Acc: 0.8900
Epoch: 018, Train Acc: 0.9050, Test Acc: 0.8700
Epoch: 019, Train Acc: 0.9225, Test Acc: 0.8850
Epoch: 020, Train Acc: 0.9150, Test Acc: 0.8950
Epoch: 021, Train Acc: 0.8250, Test Acc: